In [ ]:
import ultralytics
from ultralytics import YOLO
import numpy as np

from utils.gui_utils import UserPrompt
from utils.path_utils import join_paths

In [ ]:
################################ User Input ################################

dataset_folder = "data/yolo/CElegan-Head-5"
config_file = "data/yolo/yolo_train_config.yaml"
dataset_file = join_paths(dataset_folder, "data.yaml")

############################################################################

If Roboflow is used, copy the code produced by Roboflow to the cell below and run it to download the dataset.   
Otherwise, if the dataset is already on the system there is no need to do this step.

In [ ]:
from roboflow import Roboflow

################################ User Input ################################

rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("technion-institute-of-technology").project("celegan-head")
version = project.version(5)



dataset = version.download("yolov8", location=dataset_folder, overwrite=False)

**Load a base model**

In [ ]:
################################ User Input ################################

# can be one of the models stated at the ultraliytics documentation or a 
# path to a trained model on the disk
model_path = "yolov8s"  

############################################################################

if model_path is None:
    yolo_model_path = UserPrompt.open_file("Select YOLO model", ("pytorch file", "*.pt"))

model = YOLO(model=model_path)

**Train The Model**

In [ ]:
results = model.train(data=dataset_file, cfg=config_file)

**Export to faster format**

In [ ]:
model = YOLO(model=model_path)

out = model.export(
    format="engine",  # Use 'engine' for TensorRT (GPU), 'onnx' for ONNX (CPU/GPU).
    imgsz=384,
    half=False,  # Enables FP16 (half-precision) quantization, reducing model size and potentially speeding up inference on supported hardware.
    dynamic=False,  # Allows dynamic input sizes for ONNX and TensorRT exports, enhancing flexibility in handling varying image dimensions
    simplify=False,  # Simplifies the model graph for ONNX exports, potentially improving performance and compatibility
    verbose=False,
    batch=8,  # max supported batch size of the exported model
    workspace=8,  # max memory usage during the export process in GB
)

**Deploy Model to Roboflow**

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=model_path)